# Exercise 4. Metabolic control analysis

Tutorial: Pathway modeling using Ordinary Differential Equations

Instructor: [Veronica Llorens-Rico](veronica.llorensrico@kuleuven.vib.be)

Whole-cell Model Course; 9th April 2019

Centre for Genomic Regulation, Barcelona


## Perform MCA of the Central Carbon Metabolism model

Metabolic control analysis has a wide range of applications, interestingly in the field of metabolic engineering. By calculating Flux and Concentration control coefficients, we can determine which reactions and enzymes are more sensitive to perturbations.

------

The model was firstly described in [Chassagnole et al, 2002](http://onlinelibrary.wiley.com/doi/10.1002/bit.10288/full). An implementation in Matlab of the model was obtained from [Villaverde et al, 2015](http://gingproc.iim.csic.es/biopredynbench/documentation.html) and recoded using Python.

------

First, we load the modules, data and custom functions that are required for this exercise

In [1]:
# LOAD MODULES
# =====================================================
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from scipy.integrate import odeint
import sys
sys.path.insert(0, 'src')
import numdifftools as ndt
import warnings
warnings.filterwarnings('ignore')

# LOAD DATA AND MODEL
# =====================================================
# Import differential equations model------------------
# Import parameters
from fluxes import fluxcalc
from readData import *
import roadrunner

### Exercise 4.1. Calculate the fluxes of each reaction in the model in the steady state situation. 

As mentioned in the presentation, fluxes for each reaction are calculated as the difference between the forward and reverse reaction velocities. *J = Vf - Vr*. This is equal to the calculated reaction rates. 

In [2]:
# Load the parameters and define the steady state conditions
parameters = params('parameters.txt')
initial_cond = [0.169491,    # cdhap
                0.0988719,   # ce4p
                0.39666,     # cpg2
                2.1185,      # cpg3
                0.00813025,  # cpgp
                0.398943,    # crib5p
                0.11126,     # cribu5p
                0.273191,    # csed7p
                0.138348,    # cxyl5p
                0.597874,    # cf6p
                0.279883,    # cfdp
                0.64958,     # cg1p
                3.46767,     # cg6p
                0.22137,     # cgap
                0.0552527,   # cglex
                2.65434,     # cpep
                0.804284,    # cpg
                2.66951]     # cpyr


To determine the fluxes, i.e., the reaction rates, under these steady state conditions, we have written a dedicated function **`fluxcalc`**, located in the **`fluxes.py`** file. Take a look at the file.

The function is very similar to the one describing the system of ODEs of the model. However, this function returns the reaction rates instead of the concentrations of the different metabolites.

The function requires only two arguments: the initial conditions, and the set of parameters of the model. 

In [3]:
# Calculate the fluxe in the steady state
fluxes_eq = fluxcalc(initial_cond, parameters)
print fluxes_eq

[ 0.00308314  0.05839561  0.22379492  0.14123936  0.13954216  0.3144017
  0.32026525  0.30250574  0.30267428  0.03773648  0.18790695  0.12550097
  0.12243125  0.08383353  0.05575905  0.04543487  0.03839678  0.04542432
  0.04313566  0.00238336  0.00236527  0.00043711  0.01031493  0.0022627
  0.001037    0.01746708  0.00168411  0.00702802  0.01419221  0.05355636]


### Exercise 4.2. Calculate the matrix of Elasticities for all metabolites and all reactions in the model.

To perform MCA, we will use the Python module **RoadRunner**. This is a module specifically designed to work with SBML systems biology models, and provides tools for loading, simulating, and analyzing these models. 

This module provides dedicated functions to run Metabolic Control Analysis, facilitating the analysis. As we have previously seen in the tutorial, derivating control coefficients from branched models is not a trivial problem. 

------

First, we will load an SBML version of the model with the **RoadRunner** module.

In [4]:
# Import the SBML model of the Central Carbon Metabolism described in Chassagnole et al, 2002
model = roadrunner.RoadRunner("SBML_glycolysis.xml")

The model is imported as a *RoadRunner* object. After importing the model, we have to ensure that all the metabolite concentrations are in steady state. We can calculate the steady state of the model using the **`steadyState`** function of the **RoadRunner** module. 

We can then extract the steady state values using the function **`getSteadyStateValues`**

In [5]:
# Calculate steady state
model.steadyState()

# Get steady state values
model.getSteadyStateValues()

array([ 0.16949124,  0.09887188,  0.59787402,  0.27988254,  0.64957993,
        3.46766746,  0.2213705 ,  2.65433986,  0.80428368,  0.39665993,
        2.11850454,  0.00813025,  2.66950888,  0.39894301,  0.11126025,
        0.27319125,  0.13834824,  0.05525274])

Are these values similar to the ones that we used in the previous exercises?

-----

Now, we can calculate the enzyme elasticities with respect to all metabolites. Remember that this is calculated by derivating the reaction rate of a specific enzyme with respect to each of the metabolites, and then scaling. 

This can be done using the **`getScaledElasticityMatrix`** function from the **RoadRunner** module. 

In [6]:
# Obtain matrix of elasticities
elasticities=model.getScaledElasticityMatrix()

Before printing, it is useful to convert the elasticities array into a **Pandas** `DataFrame` object, much more adequate for visualizing here.

In [7]:
# Convert the array into a Pandas DataFrame object and print
namesMetabolites=['cdhap','ce4p','cf6p','cfdp','cg1p','cg6p','cgap','cpep','cpg', 'cpg2', 'cpg3', 'cpgp','cpyr','crib5p','cribu5p','csed7p','cxyl5p','cglex']
namesReactionRates=['vALDO','vDAHPS','vDHAP','vE4P','vENO','vEXTER','vG1PAT','vG3PDH','vG6P','vG6PDH','vGAP','vGAPDH','vGLP','vMURSyNTH','vMethSynth','vPDH','vPEP','vPFK','vPG','vPG3','vPGDH','vPGI','vPGK','vPGM','vPGP','vPK','vPPK','vPTS','vR5PI','vRIB5P','vRibu5p','vRu5P','vSED7P','vSynth1','vSynth2','vTA','vTIS','vTKA','vTKB','vTRPSYNTH','vXYL5P','vf6P','vfdP','vpepCxylase','vpg2','vpyr','vrpGluMu','vsersynth']

elasticities=pd.DataFrame(data=elasticities, index=namesReactionRates, columns=namesMetabolites)

print elasticities

                 cdhap       ce4p         cf6p       cfdp       cg1p  \
vALDO       -13.559721   0.000000     0.000000  14.322520   0.000000   
vDAHPS        0.000000   2.430627     0.000000   0.000000   0.000000   
vDHAP         1.000000   0.000000     0.000000   0.000000   0.000000   
vE4P          0.000000   1.000000     0.000000   0.000000   0.000000   
vENO          0.000000   0.000000     0.000000   0.000000   0.000000   
vEXTER        0.000000   0.000000     0.000000   0.000000   0.000000   
vG1PAT        0.000000   0.000000     0.000000   0.883437   0.831260   
vG3PDH        0.855073   0.000000     0.000000   0.000000   0.000000   
vG6P          0.000000   0.000000     0.000000   0.000000   0.000000   
vG6PDH        0.000000   0.000000     0.000000   0.000000   0.000000   
vGAP          0.000000   0.000000     0.000000   0.000000   0.000000   
vGAPDH        0.000000   0.000000     0.000000   0.000000   0.000000   
vGLP          0.000000   0.000000     0.000000   0.000000   1.00

Why is this matrix sparse?

### Exercise 4.3. Determine the Flux Control Coefficient for every reaction on the glucose uptake. 

The glucose uptake is the first reaction of the system, and it is irreversible. Which reactions are the ones increasing the flux of glucose uptake? Which reactions have a negative effect? Consider only the reactions of the network, ignore dilution effects and also the external supply of glucose.

For this exercise, we also use the **RoadRunner** module. 

The function to calculate the Flux Control Coefficients is **`getScaledFluxControlCoefficientMatrix`**

In [8]:
# Calculate and print Flux Control Coefficients, converting first to a Pandas DataFrame object
FCCmatrix=model.getScaledFluxControlCoefficientMatrix()

FCCdf=pd.DataFrame(data=FCCmatrix, index=namesReactionRates, columns=namesReactionRates)

print FCCdf

                    vALDO    vDAHPS         vDHAP          vE4P          vENO  \
vALDO        1.352008e-04  0.008773  7.084780e-06 -8.056011e-07 -1.163354e-04   
vDAHPS       1.020890e-02  0.675825 -9.431058e-05 -7.849243e-05 -3.928135e-03   
vDHAP        5.125457e-03 -0.153319  9.999493e-01 -3.632554e-05 -2.850971e-03   
vE4P         4.197224e-03 -0.133286 -3.877721e-05  9.999677e-01 -1.617587e-03   
vENO         7.345861e-05 -0.005087 -8.136718e-06 -6.193222e-06  1.390968e-04   
vEXTER      -7.968419e-07  0.000019  3.952166e-09  4.807586e-09 -3.516416e-07   
vG1PAT      -4.956065e-02 -0.294874 -9.278408e-05 -6.887740e-05 -3.941720e-03   
vG3PDH       4.382638e-03 -0.131098 -4.331091e-05 -3.106097e-05 -2.437787e-03   
vG6P         1.878075e-03 -0.053203 -1.397793e-05 -1.134603e-05  9.348567e-04   
vG6PDH       1.513588e-03 -0.042878 -1.126517e-05 -9.144048e-06  7.534244e-04   
vGAP         5.298614e-03 -0.159310 -5.078275e-05 -3.756242e-05 -2.944682e-03   
vGAPDH       1.607186e-04 -0

In this matrix, the columns indicate the 'cause', this is the reactions 'modified', and the rows the 'consequence', the effect in the fluxes.

We can select the row referring to the flux of glucose uptake to better see which are the reactions influencing it

In [9]:
# Select the row corresponding to the glucose uptake
glcUptake=FCCdf.loc['vPTS']

print glcUptake

vALDO         -7.968419e-07
vDAHPS         1.948184e-05
vDHAP          3.952166e-09
vE4P           4.807586e-09
vENO          -3.516416e-07
vEXTER         9.988067e-01
vG1PAT         8.561478e-06
vG3PDH         1.412588e-06
vG6P           3.618456e-07
vG6PDH         1.363117e-04
vGAP           5.193935e-09
vGAPDH        -5.600940e-05
vGLP           6.536501e-08
vMURSyNTH      3.268912e-06
vMethSynth    -1.651539e-06
vPDH           1.371526e-04
vPEP           7.548378e-08
vPFK           2.912486e-04
vPG            6.209641e-08
vPG3           6.012831e-08
vPGDH          4.048785e-07
vPGI           8.341962e-07
vPGK          -6.574629e-07
vPGM           3.190760e-07
vPGP           2.302921e-10
vPK            1.108677e-05
vPPK           2.818147e-05
vPTS           4.998024e-04
vR5PI          2.354164e-06
vRIB5P         3.030083e-08
vRibu5p        8.581110e-09
vRu5P         -2.542153e-06
vSED7P         3.542697e-08
vSynth1        1.451783e-05
vSynth2        3.909062e-05
vTA            9.246

Ignoring the external glucose supply (vEXTER), 

Which reactions *increase* the Flux of glucose uptake the most?

Which ones *decrease* this Flux the most?

-------

Test the summation theorem. Do the perturbations affecting the same flux add up to one?

In [10]:
# Test the summation theorem by adding all the Flux Control Coefficients of all the enzymes over the same reaction
np.sum(FCCmatrix[[1], ])

1.0

### Exercise 4.4. Determine the Concentration Control Coefficient for every reaction on the pyruvate concentration. 

Imagine that from an engineering point of view, we want to increase the concentration of pyruvate in our system. To do so, we can determine the Concentration Control Coefficients for all reactions on pyruvate, and then modify enzyme concentrations (i.e. overexpress or knock-down) accordingly.

Calculate the Concentration Control Coefficient matrix using the function **`getScaledConcentrationControlCoefficientMatrix`**.

In [11]:
# Calculate and print Concentration Control Coefficients
CCCmatrix=model.getScaledConcentrationControlCoefficientMatrix()

CCCdf=pd.DataFrame(data=CCCmatrix, index=namesMetabolites, columns=namesReactionRates)

print CCCdf

            vALDO    vDAHPS         vDHAP          vE4P      vENO    vEXTER  \
cdhap    0.005125 -0.153319 -5.065174e-05 -3.632554e-05 -0.002851  2.350716   
ce4p     0.004197 -0.133286 -3.877721e-05 -3.227563e-05 -0.001618  1.822194   
cf6p     0.001879 -0.053269 -1.399541e-05 -1.136009e-05  0.000936  0.694945   
cfdp    -0.059923 -0.295922 -9.571479e-05 -7.014013e-05 -0.005505  4.548761   
cg1p     0.004063 -0.040235 -9.895969e-06 -8.316337e-06  0.001109  0.500704   
cg6p     0.001878 -0.053203 -1.397793e-05 -1.134603e-05  0.000935  0.695793   
cgap     0.005299 -0.159310 -5.078275e-05 -3.756242e-05 -0.002945  2.366092   
cpep     0.005156 -0.152290 -4.235673e-05 -3.116398e-05  0.002657  1.404162   
cpg      0.001546 -0.043797 -1.150674e-05 -9.340134e-06  0.000770  0.572782   
cpg2     0.005155 -0.152283 -4.239282e-05 -3.119171e-05 -0.003035  1.409393   
cpg3     0.005154 -0.152256 -4.241288e-05 -3.120731e-05 -0.003033  1.412948   
cpgp     0.005145 -0.152027 -4.242346e-05 -3.121738e

In this matrix, the columns indicate the 'cause', that is the reactions 'modified', and the rows the 'consequence', the effect in the different metabolite concentrations. Select the row corresponding to the pyruvate concentration.

Get the row corresponding to the pyruvate concentration to see how it can be affected by the different reaction rates.

In [12]:
# Select the row corresponding to the pyruvate concentration
pyruvate=CCCdf.loc['cpyr']

print pyruvate

vALDO          2.941331e-05
vDAHPS        -8.780041e-04
vDHAP         -2.446819e-07
vE4P          -1.794677e-07
vENO           1.527444e-05
vEXTER         3.004851e-01
vG1PAT        -2.270785e-04
vG3PDH        -8.745449e-05
vG6P          -9.597333e-06
vG6PDH        -2.198477e-03
vGAP          -3.208403e-07
vGAPDH         2.346691e-03
vGLP          -1.733695e-06
vMURSyNTH     -8.705281e-05
vMethSynth     3.304040e-03
vPDH          -2.743852e-01
vPEP          -4.400563e-06
vPFK           6.041290e-04
vPG           -1.873916e-06
vPG3          -3.507113e-06
vPGDH          2.617612e-06
vPGI           1.253157e-06
vPGK           2.754653e-05
vPGM          -8.462944e-06
vPGP          -1.343991e-08
vPK            5.298782e-02
vPPK          -8.462508e-04
vPTS           1.503626e-04
vR5PI         -9.965257e-05
vRIB5P        -9.098921e-07
vRibu5p       -2.592854e-07
vRu5P          1.075986e-04
vSED7P        -8.634904e-07
vSynth1       -8.463621e-04
vSynth2       -7.820401e-02
vTA           -4.359

Ignoring the external glucose supply,

What is the reaction with the largest influence in *increasing* pyruvate concentration?

What is the reaction with the largest influence in *decreasing* pyruvate concentration?